# Yeast model FBA

In [ ]:
%reload_ext autoreload
%autoreload 1
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import cobra
import escher

In [ ]:
model = cobra.io.load_json_model("./models/iMM904.json")
model

In [ ]:
[model.reactions.get_by_id(r).name for r in model.medium.keys()]

In [ ]:
model.reactions.BIOMASS_SC5_notrace

The objective reaction in this case combines amino acids (building proteins?)

In [ ]:
solution = model.optimize()

In [ ]:
b = escher.Builder(map_name='iMM904.Central carbon metabolism',
                  reaction_data=solution.fluxes.to_dict())

In [ ]:
b

In [ ]:
solution.fluxes.to_dict()

In [ ]:
# set glucose intake to something "feasible"
# TODO value for ecoli, check value for yeast
model.reactions.EX_glc__D_e.bounds = (-18.5, 1)

# Aerobic vs anaerobic growth
Doing the same tests as for ecoli

## Aerobic growth

In [ ]:
model.reactions.EX_o2_e.bounds = (-1000, 1)
solution = model.optimize()
print("Growth rate: %.2f" %solution.objective_value)

In [ ]:
model.summary()

In [ ]:
b = escher.Builder(map_name='iMM904.Central carbon metabolism',
                  reaction_data=solution.fluxes.to_dict())
b

## Anaerobic growth

In [ ]:
model.reactions.EX_o2_e.bounds = (0, 1)
solution = model.optimize()
print("Growth rate: %.2f" %solution.objective_value)

In [ ]:
model.summary()

In [ ]:
b = escher.Builder(map_name='iMM904.Central carbon metabolism',
                  reaction_data=solution.fluxes.to_dict())
b

## Finding the switch

In [ ]:
objective = []
co2_flux = []
h2o_flux = []
ethanol_flux = []
pyrdc_flux = []
bounds = np.arange(-100,0,0.1)
for i in bounds:
    model.reactions.EX_o2_e.bounds = (i,0)
    sol = model.optimize()
    objective.append(sol.objective_value)
    co2_flux.append(sol.fluxes.EX_co2_e)
    h2o_flux.append(sol.fluxes.EX_h2o_e)
    ethanol_flux.append(sol.fluxes.EX_etoh_e)
    pyrdc_flux.append(sol.fluxes.PYRDC)

In [ ]:
fig, ax = plt.subplots(2,3)
ax[0][0].plot(objective, label="objective", color="red")
ax[0][1].plot(co2_flux, label="CO2", color="blue")
ax[0][2].axis("off")
ax[1][0].plot(ethanol_flux, label="ethanol", color="green")
ax[1][1].plot(h2o_flux, label="H2O", color="yellow")
ax[1][2].plot(pyrdc_flux, label="PYRDC", color="purple")
fig.suptitle("Fluxes for decreasing O2 uptake")
fig.legend()
plt.show()

## Changing the medium

In [ ]:
#glucose to fructose
with model as model:
    model.reactions.EX_glc__D_e.bounds = (0, 1)
    model.reactions.EX_fru_e.bounds = (-20, 1)
    model.reactions.EX_o2_e.bounds = (-1000, 1)
    solution = model.optimize()
    model.summary()


In [ ]:
b = escher.Builder(map_name='iMM904.Central carbon metabolism',
                  reaction_data=solution.fluxes.to_dict())
b

## Mixing sugars
Move the bounds of fructose and glucose.

In [ ]:
# fixed glucose, increase fructose
# find flow through fructose
fru_flow = []
glc_flow = []
grad = np.arange(-20,-40, -1)
with model as model:
    model.reactions.EX_glc__D_e.bounds = (-20, 1)
    for i in grad:
        model.reactions.EX_fru_e.bounds = (i, 1)
        sol = model.optimize()
        fru_flow.append(-sol.fluxes.EX_fru_e)
        glc_flow.append(-sol.fluxes.EX_glc__D_e)
    model.summary()

In [ ]:
plt.plot(grad, fru_flow, label="fructose")
plt.plot(grad, glc_flow, label="glucose")
plt.legend()

#### Additional constraint: oxygen uptake limited.

In [ ]:
# fixed glucose, increase fructose
# find flow through fructose
fru_flow = []
glc_flow = []
etoh_flow = []
o2_flow = []
grad = np.arange(0,40, 1)
with model as model:
    model.reactions.EX_o2_e.bounds = (-100, 1)
    model.reactions.EX_glc__D_e.bounds = (-20, 1)
    for i in grad:
        model.reactions.EX_fru_e.bounds = (-i, 1)
        sol = model.optimize()
        etoh_flow.append(abs(sol.fluxes.EX_etoh_e))
        fru_flow.append(abs(sol.fluxes.EX_fru_e))
        glc_flow.append(abs(sol.fluxes.EX_glc__D_e))
        o2_flow.append(abs(sol.fluxes.EX_o2_e))
    model.summary()

In [ ]:
fig, ax = plt.subplots(2,2)
ax[0][0].plot(grad, fru_flow, color="blue", label="fructose")
ax[0][1].plot(grad, glc_flow, color="red", label="glucose")
ax[1][0].plot(grad, etoh_flow, color="green", label="ethanol")
ax[1][1].plot(grad, o2_flow, color="orange", label="oxygen")
fig.legend()

In [ ]:
b = escher.Builder(map_name='iMM904.Central carbon metabolism',
                  reaction_data=sol.fluxes.to_dict())
b

#### Additional constraint: oxygen and ammonia uptake limited.

In [ ]:
# fixed glucose, increase fructose
# find flow through fructose
fru_flow = []
glc_flow = []
etoh_flow = []
o2_flow = []
grad = np.arange(0,40, 1)
with model as model:
    model.reactions.EX_o2_e.bounds = (-100, 1)
    model.reactions.EX_nh4_e.bounds = (-23, 1)
    model.reactions.EX_glc__D_e.bounds = (-20, 1)
    for i in grad:
        model.reactions.EX_fru_e.bounds = (-i, 1)
        sol = model.optimize()
        etoh_flow.append(abs(sol.fluxes.EX_etoh_e))
        fru_flow.append(abs(sol.fluxes.EX_fru_e))
        glc_flow.append(abs(sol.fluxes.EX_glc__D_e))
        o2_flow.append(abs(sol.fluxes.EX_o2_e))
    model.summary()

In [ ]:
fig, ax = plt.subplots(2,2)
ax[0][0].plot(grad, fru_flow, color="blue", label="fructose")
ax[0][1].plot(grad, glc_flow, color="red", label="glucose")
ax[1][0].plot(grad, etoh_flow, color="green", label="ethanol")
ax[1][1].plot(grad, o2_flow, color="orange", label="oxygen")
fig.legend()

In [ ]:
b = escher.Builder(map_name='iMM904.Central carbon metabolism',
                  reaction_data=sol.fluxes.to_dict())
b

## Testing essentiality of reactions

In [ ]:
essential_reactions = {''}

In [ ]:
%%time
# Check essential reactions

for i in range(10):
    for reaction in model.reactions:
        with model as model: #using the model as a context
            reaction.knock_out()
            model.optimize()
            if model.objective.value<= 0.01:
                essential_reactions.add(reaction.id)
            

In [ ]:
print(len(essential_reactions), len(model.reactions))

In [ ]:
import random
random.sample(essential_reactions, 5)

## Genes?

In [ ]:
pgi = model.reactions.PGI
gpr = pgi.gene_reaction_rule
gpr

In [ ]:
pgi.genes

In [ ]:
pgi_gene = model.genes.YBR196C
pgi_gene.reactions